<em>Ansys Chemkin</em> has many tools to obtain species properties from a <i>Chemistry Set/mechanism</i>. Some of these <em>Chemkin</em> utilities are ported to <strong>PyChemkin</strong> as <em>Chemistry</em> methods, and this tutorial will walk you through how to apply these methods to evaluate <i><u>species</u></i> thermodynamic and transport properties in <strong>PyChemkin</strong>. 

To start a <strong>PyChemkin</strong> project, you always have to import these Python packages: <code>os</code>, <code>chemkin</code>, and <code>numppy</code>. Since you are going to make some species property plots, the <code>matplotlib</code> package also needs to be imported.  

In [ ]:
import os
import chemkin as ck                      # Chemkin
import numpy as np                        # number crunching
%matplotlib inline
import matplotlib.pyplot as plt           # plotting

# check working directory
current_dir = os.getcwd()
print('current working directory: ' + current_dir)
# set verbose mode
ck.verbose = True

<h5>Set up the <em>Chemistry Set</em></h5>The GRI 3.0 methane combustion mechanism is used to create <em>Chemistry Set</em> <code>MyGasMech</code>. Remember to preprocess the <em>Chemistry Set</em> before put it to use.

In [ ]:
# set mechanism directory (the default chemkin mechanism data directory)
data_dir = ck.ansys_dir + '\\reaction\\data'
mechanism_dir = data_dir
# create a chemistry set based on GRI 3.0
MyGasMech = ck.Chemistry(label='GRI 3.0')
# set mechanism input files
# inclusion of the full file path is recommended
MyGasMech.chemfile = mechanism_dir+'\\grimech30_chem.inp'
MyGasMech.thermfile = mechanism_dir+'\\grimech30_thermo.dat'
MyGasMech.tranfile = mechanism_dir+'\\grimech30_transport.dat'
# preprocess the mechanism files
iError = MyGasMech.preprocess()

Get the element and the species symbols/names from the mechanism <code>MyGasMech</code> into lists <code>elelist</code> and <code>specieslist</code>, respectively.

In [ ]:
# extract element symbols as a list
elelist = MyGasMech.elementsymbols
# extract gas species symbols as a list
specieslist = MyGasMech.speciessymbols

Set up an optional list of species of interest.

In [ ]:
# list of gas species interested
plotspeclist = ['CH4', 'O2', 'N2']

<h5>Species elemental composition</h>

List the elemental compositions of the selected species in the <code>plotspeclist</code>. The method <code>SpeciesComposition</code> method is employed here. 

In [ ]:
# find elemental compositions of selected species
for s in plotspeclist:
    speciesID = MyGasMech.getspecindex(s)
    print('species ' + specieslist[speciesID])
    print('elemental composition')
    for elemID in range(MyGasMech.MM):
        num_elem = MyGasMech.SpeciesComposition(elemID, speciesID)
        print(f'    {elelist[elemID]:>4}: {num_elem:2d}')
    print('='*10)

<h5>Species specific heat capacity</h5>

Use the <code>SpeciesCv</code> method to obtain the specific heat capacity at constnt volume (C<sub>v</sub>) of <u>all</u> species at a given temperature. The C<sub>v</sub> values of the three species in <code>plotspeclist</code> are plotted against temperature. Note that, by default, C<sub>v</sub> is in "ergs/mole-K", a conversion factor <code>ergsperjoule</code> is applied to convert the default units to "J/mole-K". 

In [ ]:
#
# plot Cv value at different temperatures for selected gas species
#
plt.figure(figsize=(6, 6))
# temperature increment
dTemp = 20.0
# number of property data points
points = 100
# curve attributes
curvelist = ['g', 'b--', 'r:']
# create arrays
# species specific heat capacity at constant volume data
Cv = np.zeros(points, dtype=np.double)
# temperature data
T = np.zeros(points, dtype=np.double)
# start of the plotting loop #1
k = 0
# loop over the selected gas species
for s in plotspeclist:
    # starting temperature at 300K
    Temp = 300.0
    # loop over temperature data points
    for i in range(points):
        HeatCapacity = MyGasMech.SpeciesCv(Temp)
        ID = MyGasMech.getspecindex(s)
        T[i] = Temp
        # convert ergs to joules
        Cv[i] = HeatCapacity[ID]/ck.ergsperjoule
        Temp += dTemp
    plt.plot(T, Cv, curvelist[k])
    k += 1
# plot Cv versus temperature
plt.xlabel('Temperature [K]')
plt.ylabel('Cv [J/mol-K]')
plt.legend(plotspeclist, loc='upper left')
plt.show()

In [ ]:
# delete unused objects
del Cv, HeatCapacity

<h5>Species thermal conductivity</h5>

Since <code>MyGasMech</code> includes the transport data when it is preprocessed, the species transport properties such as viscosity, thermal conductivity, and diffusivity are available. You can use the <code>SpeciesCond</code> method to get the thermal conductivity of <u>all</u> species at the given temperature. Similarly, the default thermal conductivity value is in <u>cgs units</u>.    

In [ ]:
# species conductivity
kappa = np.zeros(points, dtype=np.double)
# start of the plotting loop #2
k = 0
# loop over the selected gas species
for s in plotspeclist:
    # starting temperature at 300K
    Temp = 300.0
    # loop over temperature data points
    for i in range(points):
        conductivity = MyGasMech.SpeciesCond(Temp)
        ID = MyGasMech.getspecindex(s)
        T[i] = Temp
        # convert ergs to joules
        kappa[i] = conductivity[ID]/ck.ergsperjoule
        Temp += dTemp
    plt.plot(T, kappa, curvelist[k])
    k += 1
# plot conductivity versus temperature
plt.xlabel('Temperature [K]')
plt.ylabel('Conductivity [J/cm-K-sec]')
plt.legend(plotspeclist, loc='upper left')
plt.show()

In [ ]:
# delete unused objects
del kappa, conductivity

<h5>Species binary diffusion coefficient</h5>

Without setting up a mixture, you can get the binary diffusion coeffiicents between two species only. Use the <code>SpeciesDiffusionCoeffs</code> method to evaluate the binary diffusion coefficients of all the species pairings and save them in a 2-D array <code>diffcoef</code>. The diffusion coefficient depends on temperature and pressure so you need to provide both parameters when calling <code>SpeciesDiffusionCoeffs</code>. The pressure in <em>Chemkin</em> is in "dynes/cm<sup>2</sup>" and the diffusion coefficient is in "cm<sup>2</sup>/sec".  

In [ ]:
# calculate species binary diffusion coefficients
# at 2 atm and 500K
diffcoef = MyGasMech.SpeciesDiffusionCoeffs(2.0*ck.Patm, 500.0)
ID_CH4 = MyGasMech.getspecindex(plotspeclist[0])
ID_O2 = MyGasMech.getspecindex(plotspeclist[1])
c = diffcoef[ID_CH4][ID_O2]
print(f'diffusion coefficient for {plotspeclist[0]} against {plotspeclist[1]} is {c:e} cm2/sec')

In [ ]:
# delete unused objects
del T, diffcoef